In [1]:
import os
import numpy as np

# Check if file exists and its size
if os.path.exists('super_emissions_tensor.npy'):
    size = os.path.getsize('super_emissions_tensor.npy')
    print(f"File exists and is {size} bytes")
else:
    print("File does not exist")

# Try loading with a different method
try:
    with open('super_emissions_tensor.npy', 'rb') as f:
        print("First few bytes:", f.read(10))
except Exception as e:
    print(f"Error reading file: {e}")

File exists and is 74649600000 bytes
First few bytes: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [5]:
import numpy as np

# Define the dimensions
molecules = 5  # CH4, CO2, N2O, CO2bio, GWA
emission_types = 2  # emissions and fluxes
time_steps = 288  # 3 years × 12 months
lat = 1800  # spatial grid latitude
lon = 1800  # spatial grid longitude

# Load the data in memory-efficient way using memmap
data = np.memmap('EDGAR/super_emissions_tensor.npy', dtype=np.float64, mode='r', offset=128)  # Try with offset

# Verify the numbers
print(f"Total elements in data: {len(data)}")
print(f"Total elements in proposed shape: {molecules * emission_types * time_steps * lat * lon}")

# Reshape the data
tensor = data.reshape((molecules, emission_types, time_steps, lat, lon))

# Print the shape to verify
print(f"Tensor shape: {tensor.shape}")

Total elements in data: 9331200000
Total elements in proposed shape: 9331200000
Tensor shape: (5, 2, 288, 1800, 1800)


In [5]:
tensor.shape

(5, 2, 288, 1800, 1800)

In [4]:
# Print example values and explanations
molecules = ['CH4', 'CO2', 'N2O', 'CO2bio', 'GWA']
types = ['Emissions', 'Fluxes']

# Look at center point of the grid for first timestamp
lat_center = 900
lon_center = 900
time_idx = 0

print("Sample values at the center of the spatial grid (first month):")
print("-" * 60)
for mol_idx, mol_name in enumerate(molecules):
    for type_idx, type_name in enumerate(types):
        value = tensor[mol_idx, type_idx, time_idx, lat_center, lon_center]
        print(f"{mol_name} {type_name}: {value}")

# Look at temporal variation for one location and one molecule
print("\nTemporal variation for CH4 emissions at center point:")
print("-" * 60)
# Get values for first year (12 months)
ch4_year1 = tensor[0, 0, 0:12, lat_center, lon_center]
print("Monthly values for first year:")
for month, value in enumerate(ch4_year1, 1):
    print(f"Month {month}: {value}")

# Basic statistics for each molecule
print("\nOverall statistics for each molecule:")
print("-" * 60)
for mol_idx, mol_name in enumerate(molecules):
    data = tensor[mol_idx]
    print(f"\n{mol_name}:")
    print(f"Min: {np.min(data)}")
    print(f"Max: {np.max(data)}")
    print(f"Mean: {np.mean(data)}")
    print(f"Non-zero values: {np.count_nonzero(data)/data.size*100:.2f}% of points")

Sample values at the center of the spatial grid (first month):
------------------------------------------------------------
CH4 Emissions: 0.0
CH4 Fluxes: 0.0
CO2 Emissions: 0.0
CO2 Fluxes: 0.0
N2O Emissions: 0.0
N2O Fluxes: 0.0
CO2bio Emissions: 0.0
CO2bio Fluxes: 0.0
GWA Emissions: 0.0
GWA Fluxes: 0.0

Temporal variation for CH4 emissions at center point:
------------------------------------------------------------
Monthly values for first year:
Month 1: 0.0
Month 2: 0.0
Month 3: 0.0
Month 4: 0.0
Month 5: 0.0
Month 6: 0.0
Month 7: 0.0
Month 8: 0.0
Month 9: 0.0
Month 10: 0.0
Month 11: 0.0
Month 12: 0.0

Overall statistics for each molecule:
------------------------------------------------------------

CH4:
Min: 0.0
Max: 5.574676752495975e+44
Mean: 5.234314515469449e+37
Non-zero values: 16.65% of points

CO2:
Min: 0.0
Max: 3.506869854316782e+36
Mean: 1.9652271198529748e+28
Non-zero values: 9.98% of points

N2O:
Min: 0.0
Max: 6.143620504390603e+39
Mean: 3.3251708901623096e+31
Non-zero v